In [6]:
# len(m8.data_2.read().shape)
m8.data_3.read()

{'instruments': ["['512700.HOF','159915.ZOF','588000.HOF','512690.HOF','510330.HOF','512580.HOF','512980.HOF','159870.ZOF','515700.HOF','159928.ZOF','512400.HOF','510500.HOF','512880.HOF','159940.ZOF','512680.HOF','515050.HOF','512010.HOF','159825.ZOF']"],
 'start_date': '2020-01-01',
 'end_date': '2021-12-01'}

In [3]:
# 本代码由可视化策略环境自动生成 2021年12月21日 18:41
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m10_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
#    csv_file = 'pq_data.csv'
    csv_file = '300ext_his.csv'
    coding = 'UTF-8'
    dtypes = {}
    date_cols = ['date']
    date_type = '%Y-%m-%d'
    
    dateparse = lambda x: pd.datetime.strptime(x, date_type)
    
    df = pd.read_csv(csv_file, encoding=coding, dtype=dtypes, parse_dates=date_cols, date_parser=dateparse)
    
    data = DataSource.write_df(df)
    return Outputs(data_1=data, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m10_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m5_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    # df_date = input_1.read_pickle()  # 日期
    df = input_1.read_df()  # pq 数据
    
    # start_date = pd.to_datetime(df_date['start_date'])
    # end_date = pd.to_datetime(df_date['end_date'])
    
    # 后复权
    df['open'] = df['open'] * df['adj_factor']
    df['high'] = df['high'] * df['adj_factor']
    df['low'] = df['low'] * df['adj_factor']
    df['close'] = df['close'] * df['adj_factor']
    del df['adj_factor']
    
    # 清理 df
    df['date'] = pd.to_datetime(df['date'])
    # df = df[(df['date'] > start_date) & (df['date'] < end_date)]    
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'HOF' if x[-2:] == 'SH' else x)
    df['instrument'] = df['instrument'].apply(lambda x: x[:7]+'ZOF' if x[-2:] == 'SZ' else x)
    df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
    
    # 输出准备
    df_1 = df[['instrument','date','open','high','low','close', 'volume', 'amount']]
    df_2 = df[['instrument','date','w_score','bias_cs','bias_sm','bias_ml','bias_cs_dd','bias_sm_dd','bias_ml_dd','cs_diff1','sm_diff1','ml_diff1','cs_diff2','sm_diff2','ml_diff2']]
    df_3 = {'instruments': ["['"+"','".join(set(df['instrument'].to_list()))+"']"], 'start_date': df.iloc[0]['date'], 'end_date': df.iloc[-1]['date']}
    
    data_1 = DataSource.write_df(df_1)  # 行情数据，DataFrame对象采用 write_df
    data_2 = DataSource.write_df(df_2)  # 因子数据
    data_3 = DataSource.write_pickle(df_3)  # 代码列表，list或字典等对象采用 write_pickle
    return Outputs(data_1=data_1, data_2=data_2, data_3=data_3)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m5_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m9_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df_his = input_1.read_df()  # 行情数据
    df_factor = input_2.read_df()  # 因子数据
    data_dict = input_3.read_pickle()  # 日期\代码列表
    
    start_date_str = '2017-07-10'  # 回测的开始时间
    end_date_str = '2020-01-01'  # 回测的结束时间
    
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    # 行情数据，由于要计算加权协方差，需要前推 50 天
    df_his = df_his[(df_his['date'] >= start_date) & (df_his['date'] <= end_date)]
    df_factor = df_factor[(df_factor['date'] >= start_date) & (df_factor['date'] <= end_date)]  # 因子数据
    data_dict['start_date'] = start_date_str
    data_dict['end_date'] = end_date_str
    
    data_1 = DataSource.write_df(df_his)  # 行情数据，DataFrame对象采用 write_df
    data_2 = DataSource.write_df(df_factor)  # 因子数据
    data_3 = DataSource.write_pickle(data_dict)  # 代码列表，list或字典等对象采用 write_pickle
    return Outputs(data_1=data_1, data_2=data_2, data_3=data_3)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m9_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m8_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    df_his = input_1.read_df()  # 行情数据
    df_factor = input_2.read_df()  # 因子数据
    data_dict = input_3.read_pickle()  # 日期\代码列表
    
    start_date_str = data_dict['start_date']  # 回测的开始时间
    end_date_str = data_dict['end_date']  # 回测的结束时间
    
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)

    his_start_date = (pd.to_datetime(start_date) - datetime.timedelta(days=150)).strftime('%Y-%m-%d')
    
    # 行情数据，由于要计算加权协方差，需要前推 150 天
    df_his = df_his[(df_his['date'] >= his_start_date) & (df_his['date'] <= end_date)]
    df_factor = df_factor[(df_factor['date'] >= start_date) & (df_factor['date'] <= end_date)]  # 因子数据

    df_his.index = range(len(df_his))
    df_factor.index = range(len(df_factor))
    data_dict = {'instruments': ["['"+"','".join(set(df_his['instrument'].to_list()))+"']"], 'start_date': start_date_str, 'end_date': end_date_str}
    
    data_1 = DataSource.write_df(df_his)  # 行情数据，DataFrame对象采用 write_df
    data_2 = DataSource.write_df(df_factor)  # 因子数据
    data_3 = DataSource.write_pickle(data_dict)  # 代码列表，list或字典等对象采用 write_pickle
    return Outputs(data_1=data_1, data_2=data_2, data_3=data_3)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m8_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m4_initialize_bigquant_run(context):
    import numpy as np
    import pandas as pd
    from scipy.optimize import minimize

    # ------------------------- pandas 配置 --------------------------- #
    pd.set_option('display.max_columns', None)  # 最多所有列
    pd.set_option('display.max_rows', None)  # 显示所有行
    # pd.set_option('display.float_format', lambda x: '%.6f'%x) # #显示小数位数两位
    pd.set_option('display.width', 1024)  # 显示宽度
    # pd.set_option('precision', 1)   # 设置显示数值的精度

    import warnings
    warnings.filterwarnings('ignore')  # 关闭运行时的警告
    np.set_printoptions(linewidth=100, suppress=True)  # 打印numpy时设置显示宽度，并且不用科学计数法显示
    # print('载入库完成')
    # ---------------------------------------------------------------- #

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.00001, sell_cost=0.00001, min_cost=0.2))

    # 初始仓位
    context.original_position = 0

    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()
    # print('context.ranker_prediction: \n', context.ranker_prediction)

    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    context.stock_count = 6

    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金
    # [0.33916020527361607, 0.2139862647345275, 0.16958010263680803, 0.14606834984270645, 0.13120507751234176]
    # context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, context.stock_count)])
    # print(context.stock_weights)

    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 5
    context.options['staging_times'] = 4  # 建仓次数
    context.options['staging_between_days'] = 3  # 建仓期间隔日
    context.options['staging_days'] = 0  # 建仓日计数

    from zipline.finance.slippage import SlippageModel
    class FixedPriceSlippage(SlippageModel):
        def process_order(self, data, order, bar_volume=0, trigger_check_price=0):
            if order.limit is None:
                price_field = self._price_field_buy if order.amount > 0 else self._price_field_sell
                price = data.current(order.asset, price_field)
            else:
                price = data.current(order.asset, self._price_field_buy)
            # 返回希望成交的价格和数量
            return (price, order.amount)

    # 设置price_field,默认是开盘买入，收盘卖出
    context.fix_slippage = FixedPriceSlippage(price_field_buy='open', price_field_sell='close')
    context.set_slippage(us_equities=context.fix_slippage)


def get_cov_mat(df=pd.DataFrame(), days=20):
    """
    基于代码获取历史数据的年化协方差矩阵

    :param df: 历史回测数据
    :param end_date: 计算的时点日期
    :param days: 根据时点日期，向前推多少天，获得截断回测数据的起始日期，计算加权的协方差矩阵
    :return: cov 最后一天的协方差矩阵
             rtn 最后一天的对数年化收益率
    """
    # print(df)
    df.fillna(method='ffill', inplace=True)
    # df['rtn'] = df.groupby('instrument')['close'].apply(lambda x: np.log(x / x.shift(1)))  # 计算对数日收益率
    df = np.log(df.pct_change() + 1)  # 计算对数日收益率
    # print(df)
    # df = pd.pivot_table(df, index=[u'date'], columns=[u'instrument'], values=[u'rtn'])
    df.fillna(0, inplace=True)
    # print(df)
    rtn = df.iloc[-1] * 252
    df_cov = df.ewm(days).cov() * 252  # 一年大概252个交易日，采取年化是业内通用做法
    # print(df_cov)
    cov = df_cov.loc[df_cov.index[-1][0]]  # 最后一天的协方差
    # cov.index = cov.index.droplevel()
    # print(cov)
    # print(rtn)
    return cov, rtn


def get_before_trade_day(date='2021-12-02', days=1) -> str:
    """
    获取指定日期前的交易日日期

    :param date: 需要往前推算的基准日期
    :param days: 需要往前推算的天数
    :return: 返回往前推算的日期，格式为字符串 str
    """
    past_datetime = datetime.datetime.strptime(date, "%Y-%m-%d") - datetime.timedelta(days=days)
    past_str = past_datetime.strftime("%Y-%m-%d")
    return past_str


def get_weight_scipy_opt(cov_mat, exp_rtn, method, wts_adjusted) -> pd.Series:
    """
    输入协方差矩阵，得到不同优化方法下的权重配置

    :param cov_mat: DataFrame 格式的协方差收益矩阵，index 和 column 均为资产名称
    :param exp_rtn: 收益率的期望
    :param method: 优化方法，可选的有 min_var、risk_parity、max_div、mean_weights、mean_vol
    :param wts_adjusted: 权重调整
    :return: pd.Series  index为资产名，values为weight
    """
    #     if not valudate_input_type(cov_mat, pd.DataFrame):
    #         return
    from scipy.optimize import minimize
    omega = np.matrix(cov_mat.values)  # 协方差矩阵

    def fun1(x):
        # 最小方差
        #         return np.matrix(x) * omega * np.matrix(x).T
        return np.dot(x.T, np.dot(cov_mat, x))  # 这里的风险是一个对投资组合风险的总衡量

    def fun2(x):
        # 风险平价
        tmp = (omega * np.matrix(x).T).A1
        #         risk = x * tmp
        risk = x * tmp / np.sqrt(np.matrix(x) * omega * np.matrix(x).T).A1[0]
        delta_risk = [sum((i - risk) ** 2) for i in risk]
        #         print(sum(delta_risk))
        return sum(delta_risk)

    def fun3(x):
        # 最大风险分散比
        x = np.asmatrix(x)
        cov = np.asmatrix(omega)
        vol = np.diagonal(cov)
        w_vol = np.sum(np.multiply(x, vol))
        port_vol = np.sqrt((x * cov * x.T)[0, 0])
        return -w_vol / port_vol

    # 初始值 + 约束条件
    x0 = np.ones(omega.shape[0]) / omega.shape[0]
    bnds = tuple((0, None) for x in x0)
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},)  # 总权重约束
    cons = cons + ({'type': 'ineq', 'fun': lambda x: x},)  # 仅做多约束
    cons = cons + ({'type': 'ineq', 'fun': lambda x: 0.1 - x},)  # 单仓位约束不高于20%
    options = {'disp': False, 'maxiter': 1000, 'ftol': 1e-20}

    # 执行优化解算
    if method in ['min_var', 'risk_parity', 'max_div']:
        if method == 'min_var':  # 组合方差最小时权重
            #             print('try minimize...')
            res = minimize(fun1, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
        elif method == 'risk_parity':  # 风险平价
            res = minimize(fun2, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
        elif method == 'max_div':  # 最大分散比
            bnds = [(max(0, x[0]), x[1]) for x in bnds]
            res = minimize(fun3, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
        #         print(res)
        if not res['success']:
            print('有多解存在，返回次优解，调仓方案需人工斟酌')
        wts = pd.Series(index=cov_mat.index, data=np.array(np.round(res['x'], 3)).flatten())  # 大部分情况下无最优解，只能给出次优解
    else:
        if method == 'mean_weights':  # 平均分配权重
            res = np.array([1.0 / exp_rtn.shape[1]] * exp_rtn.shape[1])
        elif method == 'mean_vol':  # 等波动率
            res = 1 / np.sqrt(exp_rtn.std())
            res = res / res.sum()
        wts = pd.Series(index=cov_mat.index, data=res)

    # 权重调整
    if wts_adjusted:
        wts = wts[wts >= 0.02]
        return wts / wts.sum() * 1.0
    elif not wts_adjusted:
        return wts

# 回测引擎：每日数据处理函数，每天执行一次
def m4_handle_data_bigquant_run(context, data):
    """
    策略交易逻辑：

    --- 建仓期：---
    分4次建仓，每次建仓期间隔 3 天
    建仓组合按因子排序，取前 5 只
    建仓期除非风控和止盈止损，仅做买入

    --- 非建仓期：---
    卖出：
    如果单只盈利超过 9%，减仓 1/4
    如果单只亏损超过 -5%，减仓 1/4
    如果大盘连续 3 日亏损超过 -10%，全部减仓 1/2

    调仓：
    每天基于因子排序，卖出排名靠后 5 只，买入排名靠前 5 只
    靠后的卖出名单，为全部卖出
    根据持仓进行筛选，实际调仓名单 <= 10 只
    将调仓计算出最优组合权重，根据当前持仓，进行对应调仓

    :param context: 上下文对象
                    context.symbol              标的对象
                    context.get_open_orders()   订单对象
                    context.portfolio           账户对象
                    context.portfolio.positions 持仓对象
    :param data:    数据对象
                    data.current_dt                                     当天日期
                    data.current(assets, fields)                        当天股票数据
                    data.history(assets, fields, bar_count, frequency)  历史股票数据
                    data.can_trade(asset)                               当天可否交易
    :return:
    """
    # 变量申明部分
    stock_sold = []  # 记录卖出的股票，防止多次卖出出现空单
    current_hold_stop_days_stock = []  # 当前存在固定天数需要卖出的股票
    today = data.current_dt.strftime('%Y-%m-%d')  # 获取当日日期
    current_stop_win_stock = []
    current_stop_loss_stock = []

    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[context.ranker_prediction['date'] == today]
    # print(f'因子打分表: \n {ranker_prediction}')

    # 获取当前的持仓信息
    """
    amount: integer 持仓数量
    cost_basis: float 持仓成本价
    last_sale_price: float 股票在交易时间的最后价格，等价于收盘价
    sid: integer 持仓股票的ID
    last_sale_date: Timestamp 最新日期
    asset: equity 标的
    """
    positions = {e.symbol: p.amount * p.last_sale_price for e, p in context.portfolio.positions.items()}  # [持仓代码: 持仓市值]
    positions_cost = {e.symbol: p.cost_basis for e, p in context.portfolio.positions.items()}  # [持仓代码: 持仓成本]
    positions_last_trade_date = {e.symbol: p.last_sale_date for e, p in context.portfolio.positions.items()}  # 获取各只股票的建仓日期
    cash = context.portfolio.cash  # 现金

    """
    cash: float 账户现金
    start_date: datetime 交易账户对象统计开始时间
    end_date: datetime交易账户对象统计结束时间
    starting_cash: float 交易账户初始金额
    portfolio_value: float 账户总价值（包括持仓市值+现金）
    positions_value: float 持仓市值
    positions: dictionary 持仓
    positions_exposure: float 持仓风险暴露
    capital_used: float 账户买卖所消耗的净资产，为正时代表花费
    pnl: float 持仓收益
    returns: float账户累计收益，比如10%返回的是0.1
    """
    print(f'当前账户：\n现金：{context.portfolio.cash}，账户价值：{context.portfolio.portfolio_value}，持仓市值：{context.portfolio.positions_value}，仓位：{np.round(100*context.portfolio.positions_value/context.portfolio.portfolio_value, 0)}%')
    print(f'当前持仓: \n{positions}')

    """
    大盘风控，如果连续
    """
    # ----------------------START: 大盘风控代码---------------------------
    # 大盘风控模块，读取风控数据
    benckmark_risk = context.benckmark_risk.loc[today]

    # 当risk为1时，市场有风险，全部平仓，不再执行其它操作
    if benckmark_risk > 0:
        for instrument in positions.keys():
            context.order_target_percent(context.symbol(instrument), positions[instrument]/context.portfolio.positions_value/2)
        print('大盘风控止损触发，卖出半仓头寸')
        return
    # -----------------------END: 大盘风控代码---------------------------

    """
    1. 资金分配
    平均持仓时间是 hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    实际操作中，会存在一定的买入误差，所以在前 hold_days 天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的 1.5 倍）
    """
    is_staging = context.trading_day_index < context.options['staging_times'] * context.options['staging_between_days']  # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['staging_times']  # 建仓期使用资金
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)  # 如果最后一期，使用剩余的所有资金
    # cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    #    print(f"资金分配{cash_avg}, {cash_for_buy}, {cash_for_sell}")

    """
    2. 根据需要加入移动止赢止损模块
    """
    # ------------------------START: 止赢止损模块(含建仓期)---------------
    if len(positions):
        for instrument in positions.keys():
            stock_cost = positions_cost[instrument]
            stock_market_price = data.current(context.symbol(instrument), 'price')

            # 赚 9% 且为可交易状态就止盈
            if stock_market_price / stock_cost - 1 >= 0.09 and data.can_trade(context.symbol(instrument)):
                # print(f'9%止盈卖出计划：{instrument}')
                context.order_target_percent(context.symbol(instrument), positions[instrument]/context.portfolio.positions_value/4)
                # cash_for_sell -= positions[instrument]
                current_stop_win_stock.append(instrument)

            # 亏 5% 并且为可交易状态就止损
            if stock_market_price / stock_cost - 1 <= -0.05 and data.can_trade(context.symbol(instrument)):
                # print(f'-5%止损卖出计划：{instrument}')
                context.order_target_percent(context.symbol(instrument), positions[instrument]/context.portfolio.positions_value/4)
                # cash_for_sell -= positions[instrument]
                current_stop_loss_stock.append(instrument)

        if len(current_stop_win_stock) > 0:
            print('9% 止盈股票列表', current_stop_win_stock)
            stock_sold += current_stop_win_stock

        if len(current_stop_loss_stock) > 0:
            print('-5% 止损股票列表', current_stop_loss_stock)
            stock_sold += current_stop_loss_stock
    # --------------------------END: 止赢止损模块--------------------------

    """
    3. 调仓；
    """
    # --------------------------START：调仓日，包含建仓期和轮动期的交易)-----------
    # print(f'因子排序表: \n{ranker_prediction}')
    sell_instruments = [k for k in reversed((ranker_prediction['instrument'][-context.stock_count:][ranker_prediction['instrument'].apply(lambda x: x in positions)]).to_list())]
    buy_instruments = [k for k in ranker_prediction['instrument'].to_list()][:context.stock_count]
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument

    # 属于建仓期，且未到建仓日
    if is_staging and context.options['staging_days'] < context.options['staging_between_days']:
        context.options['staging_days'] += 1
        return

    # 属于建仓期，且进入建仓日
    if is_staging and context.options['staging_days'] == context.options['staging_between_days'] and len(buy_instruments):
        print(f'建仓期建仓日，基于因子排序头部买入计划：{buy_instruments}')
        buy_instruments_symbol_list = [context.symbol(k) for k in buy_instruments]
        df_his = data.history(buy_instruments_symbol_list, 'close', 20, '1d')
        try:
            cov, rtn = get_cov_mat(df=df_his, days=20)
            wts = get_weight_scipy_opt(cov, rtn, method='risk_parity', wts_adjusted=True)  # min_var、risk_parity、mean_weights、mean_vol、max_div
        except Exception as err:
            print(err)
            return
        else:
            if len(wts):
                print('权重计划：\n', wts)
                for instrument, position_percent in wts.items():  # 按照权重方案，完成调仓
                    try:
                        context.order_target_percent(instrument, position_percent / context.options['staging_times'])
                    except Exception as err:
                        print(f'{instrument}调仓失败, {err}')
        context.options['staging_days'] = 0
        return

    # 不是建仓期，进入调仓日
    if not is_staging and data.current_dt.weekday() + 1 == 5:
        print('今天星期五，执行轮仓操作')
        """
        生成卖出订单，对持仓的股票，按机器学习算法预测的排序末位淘汰
        """
        if len(sell_instruments):
            # 反向排序，排位从低到高，取出持仓中包含需要卖出的个股代码
            print(f'因子排序末位卖出计划：{sell_instruments}')
            for instrument in sell_instruments:
                # 防止多个止损条件同时满足，出现多次卖出产生空单
                if instrument in stock_sold:
                    continue

                # 卖出
                context.order_target(context.symbol(instrument), 0)
                # cash_for_sell -= positions[instrument]
                stock_sold.append(instrument)

        """
        生成轮仓买入订单，按预测的排序，买入前面的 stock_count 只股票
        """
        # 正向排序，排位从高到低，取出应买入的个股代码，并与当前持仓的股票合并
        stock_hold_now = [e.symbol for e in context.portfolio.positions]  # [持仓代码]
        # print(stock_hold_now)
        # print([k for k in ranker_prediction['instrument'].to_list()][:context.stock_count])
        if len(stock_hold_now):
            buy_instruments = set([k for k in ranker_prediction['instrument'].to_list()][:context.stock_count] + stock_hold_now)
        else:
            buy_instruments = [k for k in ranker_prediction['instrument'].to_list()][:context.stock_count]

        print(f'因子排序头部买入计划加现有持股的共同组合：{buy_instruments}')
        if len(buy_instruments):
            buy_instruments_symbol_list = [context.symbol(k) for k in buy_instruments]
            # print(buy_instruments_symbol_list)
            df_his = data.history(buy_instruments_symbol_list, fields='close', bar_count=20, frequency='1d')
            # print(df_his)
            try:
                cov, rtn = get_cov_mat(df=df_his, days=20)
                wts = get_weight_scipy_opt(cov, rtn, method='risk_parity', wts_adjusted=True)  # min_var、risk_parity、mean_weights、mean_vol、max_div
            except Exception as err:
                print(err)
                return
            else:
                if len(wts):
                    print('权重计划：\n', wts)
                    for instrument, position_percent in wts.items():  # 按照权重方案，完成调仓
                        try:
                            context.order_target_percent(instrument, position_percent)
                        except Exception as err:
                            print(f'{instrument}调仓失败, {err}')




        #
        # for instrument in positions.keys():
        #     # 如果上面的止盈止损已经卖出过了，就不要重复卖出以防止产生空单
        #     if instrument in stock_sold:
        #         continue
        #
        #     # 今天和上次交易的时间相隔 hold_days 就全部卖出，也可以换成自己需要的天数，比如 datetime.timedelta(5)
        #     # print(data.current_dt, positions_last_trade_date[instrument])
        #     if data.current_dt - positions_last_trade_date[instrument] >= datetime.timedelta(context.options['hold_days']) and data.can_trade(context.symbol(instrument)):
        #         context.order_target_percent(context.symbol(instrument), 0)
        #         current_hold_stop_days_stock.append(instrument)
        #         cash_for_sell -= positions[instrument]
        # if len(current_hold_stop_days_stock) > 0:
        #     print(today, '固定持仓天数卖出计划', current_hold_stop_days_stock)
        #     stock_sold += current_hold_stop_days_stock
    # -------------------------  END: 调仓日判断-----------------------

    print('当日交易结束\n')

# 回测引擎：准备数据，只执行一次
def m4_prepare_bigquant_run(context):
    """
    在数据准备函数中一次性计算每日的大盘风控条件相比于在主函数中每日计算风控条件可以提高回测速度

    :param context:
    :return:
    """
    # -------------------------------------- START:大盘风控数据 -------------------------------------
    # 多取50天的数据便于计算均值(保证回测的第一天均值不为 NaN 值)，
    # 其中 context.start_date 和 context.end_date 是回测指定的起始时间和终止时间
    start_date = (pd.to_datetime(context.start_date) - datetime.timedelta(days=50)).strftime('%Y-%m-%d')

    df = DataSource('bar1d_index_CN_STOCK_A').read(start_date=start_date, end_date=context.end_date, fields=['close'])
    benckmark_data = df[df.instrument == '000300.HIX']

    # 计算上证指数 5 日涨幅
    benckmark_data['ret5'] = benckmark_data['close'] / benckmark_data['close'].shift(5) - 1

    # 计算大盘风控条件，如果 5 日涨幅小于 -10% 则设置风险状态 risk 为1，否则为 0
    benckmark_data['risk'] = np.where(benckmark_data['ret5'] < -0.1, 1, 0)

    # 设置日期为索引
    benckmark_data.set_index('date', inplace=True)

    # 把风控序列输出给全局变量 context.benckmark_risk
    context.benckmark_risk = benckmark_data['risk']
    # -------------------------------------- END:大盘风控数据 -------------------------------------

# 回测引擎：准备数据，每天执行一次
def m4_before_trading_start_bigquant_run(context, data):
    # 获取涨跌停状态数据
    df_price_limit_status = context.ranker_prediction.set_index('date')
    today = data.current_dt.strftime('%Y-%m-%d')

    # 得到当前未完成订单
    for orders in get_open_orders().values():

        # 循环，撤销订单
        for _order in orders:
            instrument = str(_order.sid.symbol)
            try:
                # 判断如果当日涨停，则取消卖单
                if df_price_limit_status[df_price_limit_status.instrument == instrument].price_limit_status_0.ix[today] > 2 and _order.amount < 0:
                    cancel_order(_order)
                    print(today, '尾盘涨停取消卖单', instrument)
            except:
                continue

    print('*'*180, '\n当前交易日:', today)
    pass


m1 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
w_score
bias_cs
bias_sm
bias_ml
bias_cs_dd
bias_sm_dd
bias_ml_dd
cs_diff1
sm_diff1
ml_diff1
cs_diff2
sm_diff2
ml_diff2"""
)

m10 = M.cached.v3(
    run=m10_run_bigquant_run,
    post_run=m10_post_run_bigquant_run,
    input_ports='',
    params='',
    output_ports=''
)

m5 = M.cached.v3(
    input_1=m10.data_1,
    run=m5_run_bigquant_run,
    post_run=m5_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m9 = M.cached.v3(
    input_1=m5.data_1,
    input_2=m5.data_2,
    input_3=m5.data_3,
    run=m9_run_bigquant_run,
    post_run=m9_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m6 = M.auto_labeler_on_datasource.v1(
    input_data=m9.data_1,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -5) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

# 过滤掉一字涨停的情况 (设置label为NaN，在后续处理和训练中会忽略NaN的label)
where(shift(high, -1) == shift(low, -1), NaN, label)
""",
    drop_na_label=True,
    cast_label_int=True,
    date_col='date',
    instrument_col='instrument',
    user_functions={}
)

m7 = M.standardlize.v9(
    input_1=m9.data_2,
    standard_func='ZScoreNorm',
    columns_input="""w_score
bias_cs
bias_sm
bias_ml
bias_cs_dd
bias_sm_dd
bias_ml_dd
cs_diff1
sm_diff1
ml_diff1
cs_diff2
sm_diff2
ml_diff2"""
)

m2 = M.join.v3(
    data1=m6.data,
    data2=m7.data,
    on='date,instrument',
    how='inner',
    sort=True
)

m3 = M.dropnan.v1(
    input_data=m2.data
)

m12 = M.stock_ranker_train.v6(
    training_ds=m3.data,
    features=m1.data,
    test_ds=m3.data,
    learning_algorithm='排序',
    number_of_leaves=30,
    minimum_docs_per_leaf=1000,
    number_of_trees=20,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    data_row_fraction=1,
    plot_charts=True,
    ndcg_discount_base=1,
    m_lazy_run=False
)

m11 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2020-01-01'),
    end_date=T.live_run_param('trading_date', '2021-12-01'),
    market='CN_FUND',
    instrument_list='',
    max_count=0
)

m8 = M.cached.v3(
    input_1=m5.data_1,
    input_2=m5.data_2,
    input_3=m11.data,
    run=m8_run_bigquant_run,
    post_run=m8_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m13 = M.stock_ranker_predict.v5(
    model=m12.model,
    data=m8.data_2,
    m_lazy_run=False
)

m4 = M.trade.v4(
    instruments=m8.data_3,
    options_data=m13.predictions,
    history_ds=m8.data_1,
    start_date='',
    end_date='',
    initialize=m4_initialize_bigquant_run,
    handle_data=m4_handle_data_bigquant_run,
    prepare=m4_prepare_bigquant_run,
    before_trading_start=m4_before_trading_start_bigquant_run,
    volume_limit=0,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='真实价格',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.HIX'
)


[2021-12-21 15:46:58.787499] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-12-21 15:46:58.828528] INFO: moduleinvoker: 命中缓存

[2021-12-21 15:46:58.830336] INFO: moduleinvoker: input_features.v1 运行完成[0.042906s].

[2021-12-21 15:46:58.848954] INFO: moduleinvoker: cached.v3 开始运行..

[2021-12-21 15:46:58.858199] INFO: moduleinvoker: 命中缓存

...

[2021-12-21 15:47:53.154470] INFO: Performance: Simulated 464 trading days out of 464.

[2021-12-21 15:47:53.156291] INFO: Performance: first open: 2020-01-02 09:30:00+00:00

[2021-12-21 15:47:53.157426] INFO: Performance: last close: 2021-12-01 15:00:00+00:00

[2021-12-21 15:47:56.108530] INFO: moduleinvoker: backtest.v8 运行完成[55.567729s].

[2021-12-21 15:47:56.110047] INFO: moduleinvoker: trade.v4 运行完成[56.737084s].